In [52]:
%matplotlib qt

# std
import os
from os import path
import time

# 3p
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# prj
os.chdir("..")
from src import helpers, configuration, filtering, bandpower, plots, statistics as stats
from src.configuration import cfg
os.chdir('analysis-adelie')

# statistics
from scipy.stats import ttest_ind

import seaborn as sns

# plots
import matplotlib

font = {'family':'DejaVu Sans', 'weight':'normal', 'size':16}
matplotlib.rc('font', **font)

In [45]:
import mne
from mne.viz import plot_sensors_connectivity

#### General

In [46]:
def get_concat_signal(data_type, subject="adelie", config=cfg):
    dfs = [
        helpers.load_signal_data(data_type, subject=subject, recording=recording, config=config)
        for recording in range(
            len(config["paths"]["subjects"][subject]["recordings"][data_type])
        )
    ]

    for i in range(len(dfs) - 1):
        dfs[i + 1].index += dfs[i].index.max()

    concatenated = pd.concat(dfs)
    return concatenated

March recordings

In [4]:
# get baseline Adélie
baseline_adelie_pd = get_concat_signal('baseline', subject='adelie')

# get meditation Adélie
meditation_adelie_pd = get_concat_signal('meditation', subject='adelie')

# get baseline Sam
baseline_sam_pd = get_concat_signal('baseline', subject='sam')

# get meditation Sam
meditation_sam_pd = get_concat_signal('meditation', subject='sam')

May 1 recordings

In [70]:
# get baseline Raphaël
baseline_rap_pd = get_concat_signal('baseline', subject='raphael')

# get meditation Raphaël
meditation_rap_pd = get_concat_signal('meditation', subject='raphael')

# get baseline Sam 2
baseline_sam2_pd = get_concat_signal('baseline', subject='sam2')

# get meditation Sam 2
meditation_sam2_pd = get_concat_signal('meditation', subject='sam2')

May 2 recordings

In [89]:
# get baseline Raphaël 2
baseline_rap2_pd = get_concat_signal('baseline', subject='raphael2')

# get meditation Raphaël 2
meditation_rap2_pd = get_concat_signal('meditation', subject='raphael2')

# get baseline Raphaël 3
baseline_rap3_pd = get_concat_signal('baseline', subject='raphael3')

# get meditation Raphaël 3
meditation_rap3_pd = get_concat_signal('meditation', subject='raphael3')

# get baseline Sam 3
baseline_sam3_pd = get_concat_signal('baseline', subject='sam3')

# get meditation Sam 3
meditation_sam3_pd = get_concat_signal('meditation', subject='sam3')

# get baseline Arnaud
baseline_arn_pd = get_concat_signal('baseline', subject='arn')

# get meditation Arnaud
meditation_arn_pd = get_concat_signal('meditation', subject='arn')

June recordings

In [61]:
# get baseline Sam 4
baseline_sam4_pd = get_concat_signal('baseline', subject='sam4')

# get meditation Sam 4
meditation_sam4_pd = get_concat_signal('meditation', subject='sam4')

In [68]:
# This function returns a list containing all the factors of a ginven parameters n
def getFactors(n):
    # Create an empty list for factors
    factors=[];

    # Loop over all factors
    for i in range(1, n + 1):
        if n % i == 0:
            factors.append(i)

    # Return the list of factors
    return factors

In [78]:
len(meditation_sam_pd)

262468

In [79]:
getFactors(262468)

[1, 2, 4, 65617, 131234, 262468]

## Connectivity and heatmaps

In [63]:
def get_connectivity_in_band(df, band):
    len_df = len(df) // 10 * 10
    con, freqs, times, n_epochs, n_tapers = mne.connectivity.spectral_connectivity(
        df[:len_df].values.reshape((10, 20, -1)), #259 for MR
        method="coh",
        sfreq=300,
        mode="multitaper",
        faverage=True,
        fmin=(0.5, 4, 8, 12, 30),
        fmax=(4, 8, 12, 30, 50),
    )
    con_band = pd.DataFrame(con[:, :, band], columns=df.columns, index=df.columns)
    return con_band

In [48]:
band_dict = {0: "Delta", 1: "Theta", 2: "Alpha", 3: "Beta", 4: "Gamma"}

def plot_connectivity_in_band(con_band, band, subject=""):
    # con_band = get_connectivity_in_band(df, band=band)
    ax = plt.axes()
    ax.set_title(
        "Heatmap for functional connectivity in {} band | {}".format(
            band_dict[band], subject
        )
    )
    return sns.heatmap(con_band, cmap="YlOrRd", xticklabels=True, yticklabels=True, ax=ax)

In [96]:
def plot_two_connectivities_in_band(df1, df2, band, subject=""):
    con_band_1 = get_connectivity_in_band(df1, band=band)
    con_band_2 = get_connectivity_in_band(df2, band=band)
    
    fig, (ax,ax2) = plt.subplots(ncols=2)
    fig.subplots_adjust(wspace=0.1)
    
    vmin=min(con_band_1.min().min(), con_band_2.min().min())
    vmax=max(con_band_1.max().max(), con_band_2.max().max())
    
    sns.heatmap(con_band_1, cmap="YlOrRd", ax=ax, vmin=vmin, vmax=vmax, cbar=False, xticklabels=True, yticklabels=True)
    sns.heatmap(con_band_2, cmap="YlOrRd", ax=ax2, vmin=vmin, vmax=vmax, xticklabels=True, yticklabels=True) 
    # remove xticklabels and yticklabels for MR
    
    fig.suptitle("Heatmap for functional connectivity in {} band | {}".format(band_dict[band], subject))
    
    ax.set_title("Baseline")
    ax2.set_title("Meditation")

    plt.plot()

In [105]:
plot_two_connectivities_in_band(baseline_rap3_pd, meditation_rap3_pd, 0, "Raphaël3")

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..12.640s for estimation (3793 points)
    computing connectivity for the bands:
     band 1: 0.6Hz..4.0Hz (44 points)
     band 2: 4.0Hz..8.0Hz (51 points)
     band 3: 8.1Hz..11.9Hz (50 points)
     band 4: 12.0Hz..30.0Hz (228 points)
     band 5: 30.1Hz..50.0Hz (253 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    assembling connec

## Data of Adélie

### Plots of functional connectivity

#### Baseline Adélie

In [114]:
# baseline, 14 epochs
con_baseline_ad, freqs_baseline_ad, times_baseline_ad, n_epochs_baseline_ad, n_tapers_baseline_ad = mne.connectivity.spectral_connectivity(
        baseline_adelie_pd.values.reshape((14, 20, -1)), 
        method='coh', 
        sfreq=300, 
        mode='multitaper', 
        faverage=True,
        fmin=(0.5,4,8,12,30),
        fmax=(4,8,12,30,50))

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..17.400s for estimation (5221 points)
    computing connectivity for the bands:
     band 1: 0.5Hz..4.0Hz (61 points)
     band 2: 4.0Hz..8.0Hz (70 points)
     band 3: 8.0Hz..12.0Hz (69 points)
     band 4: 12.0Hz..30.0Hz (314 points)
     band 5: 30.1Hz..50.0Hz (348 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connect

Plotting five figures (five frequency bands) of baseline Adélie (ba) with connectivity between electrodes

In [36]:
info = mne.create_info(helpers.get_channels_list(cfg), sfreq=cfg['sampling_frequency'], ch_types='eeg', montage='standard_1020')

connectivity_delta_baseline_adelie = plot_sensors_connectivity(info, con_ba[:, :, 0])
connectivity_theta_baseline_adelie = plot_sensors_connectivity(info, con_ba[:, :, 1])
connectivity_alpha_baseline_adelie = plot_sensors_connectivity(info, con_ba[:, :, 2])
connectivity_beta_baseline_adelie = plot_sensors_connectivity(info, con_ba[:, :, 3])
connectivity_gamma_baseline_adelie = plot_sensors_connectivity(info, con_ba[:, :, 4])

<ipython-input-36-7d1c96197b61>:3: DeprecationWarning: Passing montage to create_info is deprecated and will be removed in 0.21, use raw.set_montage (or epochs.set_montage, etc.) instead
  info = mne.create_info(helpers.get_channels_list(cfg), sfreq=cfg['sampling_frequency'], ch_types='eeg', montage='standard_1020')


Using pyvista 3d backend.



In [108]:
con_ba_df = pd.DataFrame(con_ba[:, :, band], index=baseline_adelie_pd.columns, columns=baseline_adelie_pd.columns)
con_ma_df = pd.DataFrame(con_ma[:, :, band], index=meditation_adelie_pd.columns, columns=meditation_adelie_pd.columns)

In [148]:
band = 0
ax = plt.axes()

sns.heatmap(con_ba_df, cmap='YlOrRd', xticklabels=True, yticklabels=True, ax = ax)
ax.set_title('Heatmap for functional connectivity in Delta band, baseline | Adélie')

sns.heatmap(con_ma_df, cmap='YlOrRd', xticklabels=True, yticklabels=True, ax = ax2)
ax.set_title('Heatmap for functional connectivity in Delta band, meditation | Adélie')

Text(0.5, 1, 'Heatmap for functional connectivity in Delta band, meditation | Adélie')

#### Meditation Adélie

In [140]:
# meditation, 6 epochs
spec_meditation_adelie_pd = mne.connectivity.spectral_connectivity(meditation_adelie_pd.values.reshape((6, 20, -1)), 
                                       method='coh',
                                       sfreq=300, 
                                       mode='multitaper', 
                                       faverage=True,
                                       fmin=(0.5,4,8,12,30), 
                                       fmax=(4,8,12,30,50))

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..137.293s for estimation (41189 points)
    computing connectivity for the bands:
     band 1: 0.5Hz..4.0Hz (481 points)
     band 2: 4.0Hz..8.0Hz (549 points)
     band 3: 8.0Hz..12.0Hz (549 points)
     band 4: 12.0Hz..30.0Hz (2471 points)
     band 5: 30.0Hz..50.0Hz (2746 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]


Plotting five figures (five frequency bands) of meditation Adélie (ma) with connectivity between electrodes

In [256]:
con_ma, freqs_ma, times_ma, n_epochs_ma, n_tapers_ma = spec_meditation_adelie_pd

info = mne.create_info(helpers.get_channels_list(cfg), sfreq=cfg['sampling_frequency'], ch_types='eeg', montage='standard_1020')

connectivity_delta_meditation_adelie = plot_sensors_connectivity(info, con_ma[:, :, 0])
connectivity_theta_meditation_adelie = plot_sensors_connectivity(info, con_ma[:, :, 1])
connectivity_alpha_meditation_adelie = plot_sensors_connectivity(info, con_ma[:, :, 2])
connectivity_beta_meditation_adelie = plot_sensors_connectivity(info, con_ma[:, :, 3])
connectivity_gamma_meditation_adelie = plot_sensors_connectivity(info, con_ma[:, :, 4])

<ipython-input-256-924273967c61>:3: DeprecationWarning: Passing montage to create_info is deprecated and will be removed in 0.21, use raw.set_montage (or epochs.set_montage, etc.) instead
  info = mne.create_info(helpers.get_channels_list(cfg), sfreq=cfg['sampling_frequency'], ch_types='eeg', montage='standard_1020')


### WIP: Statistical analysis

Each 10s epoch has a functional connectivity: baseline vs meditation, for the five frequency bands

#### Delta

In [305]:
# baseline

groups = baseline_adelie_pd.groupby(pd.Grouper(freq='10s'))
spec_baseline_adelie_pd_epochs_delta = [
    mne.connectivity.spectral_connectivity(
        group.values.reshape((1,20,-1)), 
        method='coh',
        sfreq=300, 
        mode='multitaper', 
        faverage=True,
        fmin=1, 
        fmax=4
    )[0] for name, group in groups if len(group) > 2000]

spec_baseline_adelie_pd_epochs_delta = list(map(lambda x: pd.DataFrame(x.reshape(20, 20), 
                                                                       index=baseline_adelie_pd.columns, 
                                                                       columns=baseline_adelie_pd.columns), 
                                                spec_baseline_adelie_pd_epochs_delta))


Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.007s for estimation (3003 points)
    frequencies: 1.1Hz..4.0Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connect

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.017s for estimation (3006 points)
    frequencies: 1.1Hz..4.0Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity fo

In [309]:
# meditation

groups = meditation_adelie_pd.groupby(pd.Grouper(freq='10s'))
spec_meditation_adelie_pd_epochs_delta = [
    mne.connectivity.spectral_connectivity(
        group.values.reshape((1,20,-1)), 
        method='coh',
        sfreq=300, 
        mode='multitaper', 
        faverage=True,
        fmin=1, 
        fmax=4
    )[0] for name, group in groups if len(group) > 2000]

spec_meditation_adelie_pd_epochs_delta = list(map(lambda x: pd.DataFrame(x.reshape(20, 20), 
                                                                         index=meditation_adelie_pd.columns, 
                                                                         columns=meditation_adelie_pd.columns), 
                                                  spec_meditation_adelie_pd_epochs_delta))


Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.007s for estimation (3003 points)
    frequencies: 1.1Hz..4.0Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connect

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.017s for estimation (3006 points)
    frequencies: 1.1Hz..4.0Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity fo

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.980s for estimation (2995 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.017s for estimation (3006 points)
    frequencies: 1.1Hz..4.0Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity fo

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 1.0Hz..3.9Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for

Statistical analysis: ttests 

In [324]:
merged_meditation = pd.concat(spec_meditation_adelie_pd_epochs_delta, keys=range(len(spec_meditation_adelie_pd_epochs_delta))).swaplevel(0,1)
merged_baseline = pd.concat(spec_baseline_adelie_pd_epochs_delta, keys=range(len(spec_baseline_adelie_pd_epochs_delta))).swaplevel(0,1)

full_merge = pd.concat([merged_baseline, merged_meditation], keys=['baseline', 'meditation']).swaplevel(0,1)
ttest_fc_ad_delta = full_merge.groupby(level=0).apply(lambda x: x.apply(lambda y: ttest_ind(y.xs('baseline', level=1), y.xs('meditation', level=1))))

In [326]:
# apply FWER correction: 5 frequency bands
FWER_threshold = 0.05 / 5
p_values_fc_ad_delta = ttest_fc_ad_delta.applymap(lambda x: x[1])
print('The FWER corrected pvalue is', FWER_threshold)

# download to csv
p_values_fc_ad_delta[p_values_fc_ad_delta < FWER_threshold].to_csv('p_values_fc_ad_delta.csv')
p_values_fc_ad_delta[p_values_fc_ad_delta < FWER_threshold]

The FWER corrected pvalue is 0.01


,P3,C3,F3,Fz,F4,C4,P4,Cz,Pz,Fp1,Fp2,T3,T5,O1,O2,F7,F8,A2,T6,T4
A2,8.505598e-07,NaN,1.213041e-04,NaN,NaN,NaN,NaN,0.000079,0.006364,NaN,2.552060e-08,NaN,NaN,0.000275,NaN,NaN,7.978268e-07,NaN,NaN,NaN
C3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C4,5.821781e-03,0.002796,NaN,0.003706,3.898384e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cz,2.305357e-04,0.003393,1.461547e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F3,1.099128e-03,0.000166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F4,2.170220e-03,0.000137,4.688599e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F7,2.520329e-05,0.009025,1.556511e-03,0.000040,2.224649e-04,NaN,3.737152e-03,0.004718,NaN,NaN,NaN,5.714450e-04,NaN,NaN,2.114875e-04,NaN,NaN,NaN,NaN,NaN
F8,1.152325e-07,0.000127,5.310811e-05,0.000004,5.935959e-04,0.000026,3.732447e-09,NaN,0.000697,0.000001,3.175003e-06,7.648753e-06,0.000007,0.001490,2.675796e-04,4.537167e-08,NaN,NaN,NaN,NaN
Fp1,NaN,NaN,4.258833e-03,0.000208,8.308933e-06,NaN,2.695848e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fp2,1.016209e-06,0.002257,5.728540e-05,0.000003,2.127578e-03,0.001712,4.114389e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Theta

In [327]:
# baseline

groups = baseline_adelie_pd.groupby(pd.Grouper(freq='10s'))
spec_baseline_adelie_pd_epochs_theta = [
    mne.connectivity.spectral_connectivity(
        group.values.reshape((1,20,-1)), 
        method='coh',
        sfreq=300, 
        mode='multitaper', 
        faverage=True,
        fmin=4, 
        fmax=8
    )[0] for name, group in groups if len(group) > 2000]

spec_baseline_adelie_pd_epochs_theta = list(map(lambda x: pd.DataFrame(x.reshape(20, 20), index=baseline_adelie_pd.columns, columns=baseline_adelie_pd.columns), spec_baseline_adelie_pd_epochs_theta))


Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.007s for estimation (3003 points)
    frequencies: 4.1Hz..8.0Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connect

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.017s for estimation (3006 points)
    frequencies: 4.1Hz..8.0Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity fo

In [328]:
# meditation

groups = meditation_adelie_pd.groupby(pd.Grouper(freq='10s'))
spec_meditation_adelie_pd_epochs_theta = [
    mne.connectivity.spectral_connectivity(
        group.values.reshape((1,20,-1)), 
        method='coh',
        sfreq=300, 
        mode='multitaper', 
        faverage=True,
        fmin=4, 
        fmax=8
    )[0] for name, group in groups if len(group) > 2000]

spec_meditation_adelie_pd_epochs_theta = list(map(lambda x: pd.DataFrame(x.reshape(20, 20), 
                                                                         index=meditation_adelie_pd.columns, 
                                                                         columns=meditation_adelie_pd.columns), 
                                                  spec_meditation_adelie_pd_epochs_theta))


Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.007s for estimation (3003 points)
    frequencies: 4.1Hz..8.0Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connect

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.017s for estimation (3006 points)
    frequencies: 4.1Hz..8.0Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity fo

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.980s for estimation (2995 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..10.017s for estimation (3006 points)
    frequencies: 4.1Hz..8.0Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity fo

[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 190 connections
    using t=0.000s..9.987s for estimation (2997 points)
    frequencies: 4.0Hz..7.9Hz (40 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for

Statistical analysis: ttest

In [329]:
merged_meditation = pd.concat(spec_meditation_adelie_pd_epochs_theta, keys=range(len(spec_meditation_adelie_pd_epochs_theta))).swaplevel(0,1)
merged_baseline = pd.concat(spec_baseline_adelie_pd_epochs_theta, keys=range(len(spec_baseline_adelie_pd_epochs_theta))).swaplevel(0,1)

full_merge = pd.concat([merged_baseline, merged_meditation], keys=['baseline', 'meditation']).swaplevel(0,1)
ttest_fc_ad_theta = full_merge.groupby(level=0).apply(lambda x: x.apply(lambda y: ttest_ind(y.xs('baseline', level=1), y.xs('meditation', level=1))))

In [330]:
# apply FWER correction: 5 frequency bands
FWER_threshold = 0.05 / 5
p_values_fc_ad_theta = ttest_fc_ad_theta.applymap(lambda x: x[1])
print('The FWER corrected pvalue is', FWER_threshold)

# download to csv
p_values_fc_ad_theta[p_values_fc_ad_theta < FWER_threshold].to_csv('p_values_fc_ad_theta.csv')
p_values_fc_ad_theta[p_values_fc_ad_theta < FWER_threshold]

The FWER corrected pvalue is 0.01


,P3,C3,F3,Fz,F4,C4,P4,Cz,Pz,Fp1,Fp2,T3,T5,O1,O2,F7,F8,A2,T6,T4
A2,4.560397e-11,2.014696e-06,2.828431e-06,5.068389e-08,1.238792e-03,NaN,1.841831e-11,1.542720e-08,2.209675e-04,0.000566,5.832946e-10,NaN,6.162327e-08,6.045171e-08,1.599239e-05,2.687061e-05,6.271597e-13,NaN,NaN,NaN
C3,1.424648e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C4,NaN,NaN,NaN,6.872341e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cz,5.460854e-07,1.005574e-04,2.769018e-05,4.134919e-04,1.352002e-07,NaN,5.916043e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F3,2.262429e-04,5.708109e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F4,4.836389e-06,4.375280e-06,3.783604e-09,8.209371e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F7,1.960846e-05,4.053274e-04,1.306870e-03,2.982237e-05,8.954909e-06,NaN,3.836413e-07,4.798972e-05,NaN,0.005375,1.639946e-05,2.198208e-03,6.671739e-06,3.385558e-03,1.780888e-10,NaN,NaN,NaN,NaN,NaN
F8,4.154068e-09,2.888476e-04,2.407957e-08,1.323942e-07,8.634070e-10,NaN,7.338741e-08,4.069322e-07,2.380821e-03,0.000542,1.677384e-07,3.879240e-05,1.297700e-07,3.387863e-03,1.869764e-11,1.156695e-09,NaN,NaN,NaN,NaN
Fp1,NaN,3.215540e-03,8.459549e-03,1.136385e-03,3.314729e-06,NaN,6.085066e-05,5.893253e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fp2,2.402471e-09,9.017459e-05,1.069619e-06,5.614535e-07,2.799324e-05,0.001412,3.461784e-07,1.354595e-06,2.271488e-03,0.000062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data of Matthieu Ricard

In [50]:
from src.configuration import cfg_mr
baseline_mr_pd = helpers.load_signal_data('baseline', subject='mr', config=cfg_mr)
meditation_mr_pd = helpers.load_signal_data('meditation', subject='mr', config=cfg_mr)

In [58]:
plot_two_connectivities_in_band(baseline_mr_pd, meditation_mr_pd, 0, "Matthieu Ricard")

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 33411 connections
    using t=0.000s..29.997s for estimation (9000 points)
    computing connectivity for the bands:
     band 1: 0.5Hz..4.0Hz (105 points)
     band 2: 4.0Hz..8.0Hz (120 points)
     band 3: 8.0Hz..12.0Hz (120 points)
     band 4: 12.0Hz..30.0Hz (540 points)
     band 5: 30.0Hz..50.0Hz (600 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    assembling c

In [59]:
plot_two_connectivities_in_band(baseline_mr_pd, meditation_mr_pd, 1, "Matthieu Ricard")

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 33411 connections
    using t=0.000s..29.997s for estimation (9000 points)
    computing connectivity for the bands:
     band 1: 0.5Hz..4.0Hz (105 points)
     band 2: 4.0Hz..8.0Hz (120 points)
     band 3: 8.0Hz..12.0Hz (120 points)
     band 4: 12.0Hz..30.0Hz (540 points)
     band 5: 30.0Hz..50.0Hz (600 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    assembling c

In [55]:
plot_two_connectivities_in_band(baseline_mr_pd, meditation_mr_pd, 2, "Matthieu Ricard")

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 33411 connections
    using t=0.000s..29.997s for estimation (9000 points)
    computing connectivity for the bands:
     band 1: 0.5Hz..4.0Hz (105 points)
     band 2: 4.0Hz..8.0Hz (120 points)
     band 3: 8.0Hz..12.0Hz (120 points)
     band 4: 12.0Hz..30.0Hz (540 points)
     band 5: 30.0Hz..50.0Hz (600 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    assembling c

In [56]:
plot_two_connectivities_in_band(baseline_mr_pd, meditation_mr_pd, 3, "Matthieu Ricard")

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 33411 connections
    using t=0.000s..29.997s for estimation (9000 points)
    computing connectivity for the bands:
     band 1: 0.5Hz..4.0Hz (105 points)
     band 2: 4.0Hz..8.0Hz (120 points)
     band 3: 8.0Hz..12.0Hz (120 points)
     band 4: 12.0Hz..30.0Hz (540 points)
     band 5: 30.0Hz..50.0Hz (600 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    assembling c

In [57]:
plot_two_connectivities_in_band(baseline_mr_pd, meditation_mr_pd, 4, "Matthieu Ricard")

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 33411 connections
    using t=0.000s..29.997s for estimation (9000 points)
    computing connectivity for the bands:
     band 1: 0.5Hz..4.0Hz (105 points)
     band 2: 4.0Hz..8.0Hz (120 points)
     band 3: 8.0Hz..12.0Hz (120 points)
     band 4: 12.0Hz..30.0Hz (540 points)
     band 5: 30.0Hz..50.0Hz (600 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    assembling c

In [ ]:
# meditation, 4 epochs
con_mr, freqs_mr, times_mr, n_epochs_mr, n_tapers_mr = mne.connectivity.spectral_connectivity(meditation_mr_pd.values.reshape((10, 259, -1)), 
                                       method='coh', 
                                       sfreq=300, 
                                       mode='multitaper', 
                                       faverage=True,
                                       fmin=(0.5,4,8,12,30), 
                                       fmax=(4,8,12,30,50))

In [94]:
%matplotlib qt

band = 2
con_mr_df = pd.DataFrame(con_mr[:, :, band], index=meditation_mr_pd.columns, columns=meditation_mr_pd.columns)
ax = plt.axes()
sns.heatmap(con_mr_df, cmap='YlOrRd', ax = ax)

ax.set_title('Heatmap for functional connectivity in Alpha band | Matthieu Ricard')

Text(0.5, 1, 'Heatmap for functional connectivity | Matthieu Ricard')